# TP5: Clase 6

Utilizando la webcam, implementar un algoritmo de seguimiento con CamShift que:

a - Permita elegir la ROI del objeto a seguir

b - Permita cambiar la escala de la ventana (no necesariamente la orientación)


In [2]:
import cv2 as cv
import numpy as np

In [21]:
cap = cv.VideoCapture(1)
#cap.set(cv.CAP_PROP_FPS, 5)

# Se toma el primer frame del video
ret, frame = cap.read()

# Se define la ubicación inicial de la ventana de seguimiento
x, y, w, h = 300, 200, 100, 50
track_window = (x, y, w, h)

# Se establece la ROI para realizar el seguimiento
roi = frame[y:y + h, x:x + w]

# Se pasa la ROI a HSV (más estables a cambios de intensidad)
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)

# Se define una máscara para el histograma (Hue: 0..180, Saturation:60..255, Value:32..255)
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

# Se calcula el histograma sobre esa máscara (toma solo el Hue: tonalidad)
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])

# Se normaliza el histograma (entre 0 y 255 según indica el flag cv.NORM_MINMAX)
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX);

In [22]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10)

while(1):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180] ,1)
        
        # Se aplica camshift al nuevo frame
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        
        # Se dibuja la ventana sobre la imagen
        x, y, w, h = track_window
        img2 = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        
        cv.imshow('Seguimiento', img2)
        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()
cap.release()